In [21]:
# !pip install duckduckgo-search
# !pip install wikipedia
# !pip install langchainhub
# !pip install numexpr
# !pip install --upgrade --quiet  langchain-community arxiv

# 1. Basics on Using Langchain's Agents

**Note:** Chains are for stacking multiple LLM outputs (from the same or a different one), and agents are for making your AI interact with tools (using code) to get more information for creating prompts.

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.schema import HumanMessage, SystemMessage
load_dotenv()

In [6]:
llm = ChatOpenAI(temperature=0.5)

### 1.1.1 Using simple prompting with LLM

In [9]:
# Try to fin the 2023 world series champion
message = [
    SystemMessage(
        content="A user will input in a year and you will get the baseball world series champion"
    ),
    HumanMessage(
        content="2023"
    )
]

llm.invoke(message).content

### 1.1.2 Using Langchain's agent with 'Wiki' tool

In [18]:
# Example with Wikipedia tools
prompt = hub.pull("hwchase17/openai-functions-agent")

from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)
tools = [wikitool]

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "Who won 2023 world series?"})

### 1.1.3 Using Langchain's agent with your own function as a tool

In [43]:
# Another example with custom functions as tools
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # THIS IS IMPORTANT TO PROVIDE
    ]
)
model = ChatOpenAI(temperature=0.5)

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2

tools = [magic_function]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "what is the value of magic_function(3)?"})

### 1.1.4 Using Langchain's agent with multiple tools (Wiki and duck-duck-go search)

In [23]:
# Example with using multiple tools
from langchain import LLMMathChain
from langchain.agents import Tool
from langchain.tools import DuckDuckGoSearchRun

# Tool #1
llm_match_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
    func=llm_match_chain,
    name="Calculator",
    description="Useful for answering math questions. Only math questions and nothing else."
)

# Tool #2
search = DuckDuckGoSearchRun()

tools2 = [search, math_tool]

agent2 = create_openai_functions_agent(llm, tools2, prompt)

agent_executor2 = AgentExecutor(agent=agent2, tools=tools2, verbose=True)
agent_executor2.invoke({"input": "Who won the 2021 world series and how many years was it since their last year series win?"})

### 1.1.5 Using Langchain's agent with chat history

In [44]:
# Using with chat history
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "input": "what's my name?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)['output']

# 2. ReAct

The idea is to make the models do reasoning + acting + observing.
It evolved from chain-of-thought (which is just about doing a few prompts together).

Example:
Question: 'Seven Brief Lesons on Physics was written by an Italian physicist that has worked in France since what year?'


#### Standard
Answer: 1986. **(INCORRECT)**

#### Reason only
Thought: Let's think step by step. Seven Brief Lessons on Physics was written by Carlo Rovelli. Carlo Rovelli has worked in France since 1990. \ 
Answer: 1990. **(INCORRECT)**

#### Act only
Act 1: Search 'Seven Brief Lessons on Physics' \
Obs 1: Seven Brief Lessons on Physics is a short boook by the Italian physicist Carlo Rovelli. Originally published in Italian in ... 

Act 2: Lookup 'Carlo Rovelli' \
Obs 2: Seven Brief Lessons on Physics is a short book by the Italian physicst Carlo Rovelli. 

Act 3: Finish 1983 \
Answer: 1983 **(INCORRECT)**

#### ReAct
Thought 1: I need to search Seven Brief Lesons on Physics, find its author, then find when the author has worked in France since. \
Act 1: Search 'Seven Brief Lessons on Physics' \
Obs 1: Seven Brief Lessons on Physics is a short book by the Italian physicist Carlo Rovelli. Originally published ...

Thought 2: The author of Seven Brief Lessons on Physics is Carlo Rovelli. I need to search Carlo Rovelli next and find when he has worked in France since. \
Act 2: Search 'Carlo Rovelli' \
Obs 2: Carlo Rovelli is an Italian theoretical physicst and writer who has worked in Italy, the United States, and since 2000, in France. He is also currently a Distinguished Chair at the ... 

Thought 3: Carlo Rovelli has worked in France since 2000. So the answer is 2000. \
Act 3: Finish 2000

Answer: 2000  **(CORRECT)**

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.5)

## 2.1 Using a Simple Prompt

In [29]:
# Simple prompt

prompt = "How would I get from Singapore to San Francisco?"
llm.invoke(prompt).content

## 2.2 Using Chain-of-thought prompt framework

In [30]:
# Chain of thought

prompt = "Explain step by step. How would I get from Singapore to San Francisco?"
llm.invoke(prompt).content

## 2.3 Using ReAct prompt framework (manually; not with langchain's agent yet)

In [32]:
# Manual ReAct (few-shot prompting on how to do ReAct to answer; THIS IS WITHOUT ACTUALLY USING TOOLS)

question = "How old is the president of the United States?"
manual_react = f"""Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question:{question}"""

llm.invoke(manual_react).content

#### How are agents different from ReAct?
ReAct is just a framework of reasoning-action-observation. The idea is to feed a model multiple examples of this structure (few-shot prompting). You use different tools for actions and their outputs determine what is the observation. Keep in mind that there is also **ZERO-SHOT REACT**. There you simply explain to the model what components of ReAct are, without providing actual examples. \
On the other hand, using agents (in langchain) just refers to using actions. You can use actions for an action-only framework (as seen above).

# 3. Using langchain's Agents with React framework

In [45]:
# USING TOOL: arxiv
from langchain_community.utilities import ArxivAPIWrapper

arxiv = ArxivAPIWrapper()
docs = arxiv.run("1605.08386")
docs

In [35]:
# USING AGENT (WITH 'arxiv' TOOL)

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)
tools = load_tools(
    ["arxiv"],
)
prompt = hub.pull("hwchase17/react")  # THIS MAKES THE AGENT USE REACT PROMPT FRAMEWORK

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "What's the paper 1605.08386 about?",
    }
)

In [48]:
# USING TOOL: Wikipedia

from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)

wikitool.run("cat")

In [51]:
# USING AGENT (WITH 'Wikipedia' TOOL)

prompt = hub.pull("hwchase17/react")

from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import create_react_agent


api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)
tools = [wikitool]

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Who won 2023 world series?"})

In [55]:
# USING AGENT (WITH your function AS A TOOL)
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent, tool
from langchain_openai import ChatOpenAI

prompt = hub.pull("hwchase17/react")

model = ChatOpenAI(temperature=0.5)

@tool
def magic_function(input: str) -> int:
    """Applies a magic function to an input."""
    return str(int(input) + 2)

tools = [magic_function]

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "what is the value of magic_function(3)?"})